# ***Car Accident Severity***

---


## Extract Data


In [47]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [48]:
df = pd.read_csv('/content/Data-Collisions.csv')
df.head(3)

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0.0,0.0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0.0,0.0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0.0,0.0,N


In [49]:
features=['SEVERITYCODE','COLLISIONTYPE','PERSONCOUNT','VEHCOUNT','WEATHER','ROADCOND', 'LIGHTCOND','SPEEDING']

In [50]:
df=df[features]
df.head(3)

,SEVERITYCODE,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,2,Angles,2,2,Overcast,Wet,Daylight,NaN
1,1,Sideswipe,2,2,Raining,Wet,Dark - Street Lights On,NaN
2,1,Parked Car,4,3,Overcast,Dry,Daylight,NaN


In [51]:
df.shape

(13846, 8)

# Data Wrangling

In [52]:
df1 = df.isnull()
df1.head()

,SEVERITYCODE,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,True


In [53]:
for column in df1.columns.values.tolist():
  print(column)
  print(df1[column].value_counts())
  print(" ")


SEVERITYCODE
False    13846
Name: SEVERITYCODE, dtype: int64
 
COLLISIONTYPE
False    13709
True       137
Name: COLLISIONTYPE, dtype: int64
 
PERSONCOUNT
False    13846
Name: PERSONCOUNT, dtype: int64
 
VEHCOUNT
False    13846
Name: VEHCOUNT, dtype: int64
 
WEATHER
False    13703
True       143
Name: WEATHER, dtype: int64
 
ROADCOND
False    13704
True       142
Name: ROADCOND, dtype: int64
 
LIGHTCOND
False    13701
True       145
Name: LIGHTCOND, dtype: int64
 
SPEEDING
True     13086
False      760
Name: SPEEDING, dtype: int64
 


In [54]:
df['COLLISIONTYPE'].value_counts()

Parked Car    3495
Rear Ended    2568
Angles        2409
Other         2035
Sideswipe     1170
Left Turn     1041
Pedestrian     404
Cycles         271
Right Turn     171
Head On        145
Name: COLLISIONTYPE, dtype: int64

In [55]:
df.dropna(subset=['COLLISIONTYPE'], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [56]:
df['WEATHER'].value_counts()

Clear                       8222
Raining                     2436
Overcast                    2078
Unknown                      779
Other                         76
Snowing                       53
Fog/Smog/Smoke                45
Sleet/Hail/Freezing Rain       8
Blowing Sand/Dirt              5
Severe Crosswind               1
Name: WEATHER, dtype: int64

In [57]:
df["WEATHER"].replace(np.nan, "Clear", inplace=True)

In [58]:
#df['ROADCOND'].value_counts()
#df['LIGHTCOND'].value_counts()
df['SPEEDING'].value_counts()

Y    760
Name: SPEEDING, dtype: int64

In [59]:
df["ROADCOND"].replace(np.nan, "Dry", inplace=True)
df["LIGHTCOND"].replace(np.nan, "Dry", inplace=True)
df.drop('SPEEDING',axis='columns', inplace=True)
df.head(10)

,SEVERITYCODE,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,WEATHER,ROADCOND,LIGHTCOND
0,2,Angles,2,2,Overcast,Wet,Daylight
1,1,Sideswipe,2,2,Raining,Wet,Dark - Street Lights On
2,1,Parked Car,4,3,Overcast,Dry,Daylight
3,1,Other,3,3,Clear,Dry,Daylight
4,2,Angles,2,2,Raining,Wet,Daylight
5,1,Angles,2,2,Clear,Dry,Daylight
6,1,Angles,2,2,Raining,Wet,Daylight
7,2,Cycles,3,1,Clear,Dry,Daylight
8,1,Parked Car,2,2,Clear,Dry,Daylight
9,2,Angles,2,2,Clear,Dry,Daylight


In [60]:
df['SEVERITYCODE'].value_counts()

1    9483
2    4226
Name: SEVERITYCODE, dtype: int64

In [61]:
df.groupby(['COLLISIONTYPE'])['SEVERITYCODE'].value_counts(normalize=True)

COLLISIONTYPE  SEVERITYCODE
Angles         1               0.574927
               2               0.425073
Cycles         2               0.904059
               1               0.095941
Head On        1               0.510345
               2               0.489655
Left Turn      1               0.585975
               2               0.414025
Other          1               0.736609
               2               0.263391
Parked Car     1               0.946495
               2               0.053505
Pedestrian     2               0.898515
               1               0.101485
Rear Ended     1               0.536215
               2               0.463785
Right Turn     1               0.812865
               2               0.187135
Sideswipe      1               0.875214
               2               0.124786
Name: SEVERITYCODE, dtype: float64

In [62]:
Feature = df[['SEVERITYCODE','PERSONCOUNT','VEHCOUNT','WEATHER','ROADCOND', 'LIGHTCOND']]
Feature = pd.concat([Feature,pd.get_dummies(df['COLLISIONTYPE'])], axis=1)
Feature.drop(['Angles','Head On','Left Turn','Rear Ended'], axis = 1,inplace=True)
Feature.head()


,SEVERITYCODE,PERSONCOUNT,VEHCOUNT,WEATHER,ROADCOND,LIGHTCOND,Cycles,Other,Parked Car,Pedestrian,Right Turn,Sideswipe
0,2,2,2,Overcast,Wet,Daylight,0,0,0,0,0,0
1,1,2,2,Raining,Wet,Dark - Street Lights On,0,0,0,0,0,1
2,1,4,3,Overcast,Dry,Daylight,0,0,1,0,0,0
3,1,3,3,Clear,Dry,Daylight,0,1,0,0,0,0
4,2,2,2,Raining,Wet,Daylight,0,0,0,0,0,0


In [63]:
Feature.columns

Index(['SEVERITYCODE', 'PERSONCOUNT', 'VEHCOUNT', 'WEATHER', 'ROADCOND',
       'LIGHTCOND', 'Cycles', 'Other', 'Parked Car', 'Pedestrian',
       'Right Turn', 'Sideswipe'],
      dtype='object')

In [64]:
df.groupby(['WEATHER'])['SEVERITYCODE'].value_counts(normalize=True)

WEATHER                   SEVERITYCODE
Blowing Sand/Dirt         1               0.800000
                          2               0.200000
Clear                     1               0.675134
                          2               0.324866
Fog/Smog/Smoke            1               0.600000
                          2               0.400000
Other                     1               0.921053
                          2               0.078947
Overcast                  1               0.688162
                          2               0.311838
Raining                   1               0.660920
                          2               0.339080
Severe Crosswind          1               1.000000
Sleet/Hail/Freezing Rain  1               0.875000
                          2               0.125000
Snowing                   1               0.924528
                          2               0.075472
Unknown                   1               0.937099
                          2               0

In [65]:
df.groupby(['ROADCOND'])['SEVERITYCODE'].value_counts(normalize=True)

ROADCOND        SEVERITYCODE
Dry             1               0.676895
                2               0.323105
Ice             1               0.844828
                2               0.155172
Oil             2               1.000000
Other           1               0.800000
                2               0.200000
Sand/Mud/Dirt   1               0.833333
                2               0.166667
Snow/Slush      1               0.863636
                2               0.136364
Standing Water  2               0.571429
                1               0.428571
Unknown         1               0.948849
                2               0.051151
Wet             1               0.668224
                2               0.331776
Name: SEVERITYCODE, dtype: float64

In [66]:
df.groupby(['LIGHTCOND'])['SEVERITYCODE'].value_counts(normalize=True)

LIGHTCOND                 SEVERITYCODE
Dark - No Street Lights   1               0.838095
                          2               0.161905
Dark - Street Lights Off  1               0.733333
                          2               0.266667
Dark - Street Lights On   1               0.711149
                          2               0.288851
Dark - Unknown Lighting   1               1.000000
Dawn                      1               0.668571
                          2               0.331429
Daylight                  1               0.662297
                          2               0.337703
Dry                       1               0.500000
                          2               0.500000
Dusk                      1               0.676471
                          2               0.323529
Other                     1               0.611111
                          2               0.388889
Unknown                   1               0.960352
                          2               0

In [67]:
Feature = Feature[['SEVERITYCODE','PERSONCOUNT', 'VEHCOUNT', 'ROADCOND', 'LIGHTCOND', 'Cycles',
       'Other', 'Parked Car', 'Pedestrian', 'Right Turn', 'Sideswipe']]
Feature = pd.concat([Feature,pd.get_dummies(df['WEATHER'])], axis=1)
Feature.head()

,SEVERITYCODE,PERSONCOUNT,VEHCOUNT,ROADCOND,LIGHTCOND,Cycles,Other,Parked Car,Pedestrian,Right Turn,Sideswipe,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Other,Overcast,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Unknown
0,2,2,2,Wet,Daylight,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,1,2,2,Wet,Dark - Street Lights On,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,1,4,3,Dry,Daylight,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
3,1,3,3,Dry,Daylight,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,2,2,2,Wet,Daylight,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [68]:
Feature.columns

Index(['SEVERITYCODE', 'PERSONCOUNT', 'VEHCOUNT', 'ROADCOND', 'LIGHTCOND',
       'Cycles', 'Other', 'Parked Car', 'Pedestrian', 'Right Turn',
       'Sideswipe', 'Blowing Sand/Dirt', 'Clear', 'Fog/Smog/Smoke', 'Other',
       'Overcast', 'Raining', 'Severe Crosswind', 'Sleet/Hail/Freezing Rain',
       'Snowing', 'Unknown'],
      dtype='object')

In [69]:
Feature = Feature[['SEVERITYCODE','PERSONCOUNT', 'VEHCOUNT', 'LIGHTCOND', 'Cycles', 'Other',
       'Parked Car', 'Pedestrian', 'Right Turn', 'Sideswipe',
       'Blowing Sand/Dirt', 'Clear', 'Fog/Smog/Smoke', 'Other', 'Overcast',
       'Raining', 'Severe Crosswind', 'Sleet/Hail/Freezing Rain', 'Snowing',
       'Unknown']]
Feature = pd.concat([Feature,pd.get_dummies(df['ROADCOND'])], axis=1)
Feature.drop(['Standing Water'], axis = 1,inplace=True)
Feature.head()

,SEVERITYCODE,PERSONCOUNT,VEHCOUNT,LIGHTCOND,Cycles,Other,Other,Parked Car,Pedestrian,Right Turn,Sideswipe,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Other,Other,Overcast,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Unknown,Dry,Ice,Oil,Other,Sand/Mud/Dirt,Snow/Slush,Unknown,Wet
0,2,2,2,Daylight,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,2,2,Dark - Street Lights On,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,1,4,3,Daylight,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
3,1,3,3,Daylight,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2,2,2,Daylight,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [70]:
Feature.columns

Index(['SEVERITYCODE', 'PERSONCOUNT', 'VEHCOUNT', 'LIGHTCOND', 'Cycles',
       'Other', 'Other', 'Parked Car', 'Pedestrian', 'Right Turn', 'Sideswipe',
       'Blowing Sand/Dirt', 'Clear', 'Fog/Smog/Smoke', 'Other', 'Other',
       'Overcast', 'Raining', 'Severe Crosswind', 'Sleet/Hail/Freezing Rain',
       'Snowing', 'Unknown', 'Dry', 'Ice', 'Oil', 'Other', 'Sand/Mud/Dirt',
       'Snow/Slush', 'Unknown', 'Wet'],
      dtype='object')

In [71]:
Feature = Feature[['SEVERITYCODE','PERSONCOUNT', 'VEHCOUNT', 'Cycles', 'Other', 'Other',
       'Parked Car', 'Pedestrian', 'Right Turn', 'Sideswipe',
       'Blowing Sand/Dirt', 'Clear', 'Fog/Smog/Smoke', 'Other', 'Other',
       'Overcast', 'Raining', 'Severe Crosswind', 'Sleet/Hail/Freezing Rain',
       'Snowing', 'Unknown', 'Dry', 'Ice', 'Oil', 'Other', 'Sand/Mud/Dirt',
       'Snow/Slush', 'Unknown', 'Wet']]
Feature = pd.concat([Feature,pd.get_dummies(df['LIGHTCOND'])], axis=1)
Feature.drop(['Dry'], axis = 1,inplace=True)
Feature.head()

,SEVERITYCODE,PERSONCOUNT,VEHCOUNT,Cycles,Other,Other,Other,Other,Other,Other,Other,Other,Other,Other,Parked Car,Pedestrian,Right Turn,Sideswipe,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Other,Other,Other,Other,Other,Other,Other,Other,Other,Other,Overcast,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Unknown,Unknown,Ice,Oil,Other,Other,Other,Other,Other,Sand/Mud/Dirt,Snow/Slush,Unknown,Unknown,Wet,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,Other,Unknown
0,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,1,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
2,1,4,3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,1,3,3,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0


In [72]:
Feature.columns

Index(['SEVERITYCODE', 'PERSONCOUNT', 'VEHCOUNT', 'Cycles', 'Other', 'Other',
       'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other',
       'Parked Car', 'Pedestrian', 'Right Turn', 'Sideswipe',
       'Blowing Sand/Dirt', 'Clear', 'Fog/Smog/Smoke', 'Other', 'Other',
       'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other',
       'Overcast', 'Raining', 'Severe Crosswind', 'Sleet/Hail/Freezing Rain',
       'Snowing', 'Unknown', 'Unknown', 'Ice', 'Oil', 'Other', 'Other',
       'Other', 'Other', 'Other', 'Sand/Mud/Dirt', 'Snow/Slush', 'Unknown',
       'Unknown', 'Wet', 'Dark - No Street Lights', 'Dark - Street Lights Off',
       'Dark - Street Lights On', 'Dark - Unknown Lighting', 'Dawn',
       'Daylight', 'Dusk', 'Other', 'Unknown'],
      dtype='object')

In [73]:
X=np.asarray(Feature[['PERSONCOUNT', 'VEHCOUNT', 'Cycles', 'Other', 'Other', 'Other', 'Other',
       'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Parked Car',
       'Pedestrian', 'Right Turn', 'Sideswipe', 'Blowing Sand/Dirt', 'Clear',
       'Fog/Smog/Smoke', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other',
       'Other', 'Other', 'Other', 'Other', 'Overcast', 'Raining',
       'Severe Crosswind', 'Sleet/Hail/Freezing Rain', 'Snowing', 'Unknown',
       'Unknown', 'Ice', 'Oil', 'Other', 'Other', 'Other', 'Other', 'Other',
       'Sand/Mud/Dirt', 'Snow/Slush', 'Unknown', 'Unknown', 'Wet',
       'Dark - No Street Lights', 'Dark - Street Lights Off',
       'Dark - Street Lights On', 'Dark - Unknown Lighting', 'Dawn',
       'Daylight', 'Dusk', 'Other', 'Unknown']])
Y=np.asarray(Feature['SEVERITYCODE'])

In [74]:
from sklearn import preprocessing
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[-0.3804971 ,  0.03623114, -0.14200948, ..., -0.24545363,
        -0.24595435, -0.22863077],
       [-0.3804971 ,  0.03623114, -0.14200948, ..., -0.24545363,
        -0.24595435, -0.22863077],
       [ 1.04489837,  1.91764287, -0.14200948, ..., -0.24545363,
        -0.24595435, -0.22863077],
       [ 0.33220064,  1.91764287, -0.14200948, ..., -0.24545363,
        -0.24595435, -0.22863077],
       [-0.3804971 ,  0.03623114, -0.14200948, ..., -0.24545363,
        -0.24595435, -0.22863077]])

In [75]:
from sklearn.metrics import jaccard_similarity_score, f1_score, log_loss

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2)


In [78]:
print('Train set:', X_train.shape)
print('Train set:', X_test.shape)

Train set: (10967, 728)
Train set: (2742, 728)


# K Nearest Neighbor

In [79]:
from sklearn.neighbors import KNeighborsClassifier
k=25

In [80]:
neigh= KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train)
neigh
Kyhat= neigh.predict(X_test)
Kyhat[0:5]

array([1, 1, 2, 1, 1])

# Evaluation Metrics

In [81]:
jaccard_similarity_score(y_test, Kyhat)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)


0.7370532458059811

In [82]:
f1_score(y_test, Kyhat, average='macro')

0.6490627831864573